In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import joblib

In [2]:
tr_tr_new = joblib.load('../joblib/tr_tr_new.joblib')

In [3]:
te_tr_new = joblib.load('../joblib/te_tr_new.joblib')

In [5]:
def optimize_dtypes(df):
    for col in df.columns:
        col_dtype = df[col].dtype
        if col_dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif col_dtype == 'int64' or col_dtype == 'int32':
            df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

# Apply the optimization function on your datasets
tr_tr_new = optimize_dtypes(tr_tr_new)
te_tr_new = optimize_dtypes(te_tr_new)

In [6]:
# Identify categorical features. CatBoost can handle them natively.
cat_var = ['ProductCD',
           'card6', 'card2', 'card5', 'card1', 'card4', 'card3',
           'addr2', 'addr1',
           'P_emaildomain',
           'R_emaildomain',
           'M6', 'M7', 'M8', 'M4', 'M9', 'M1', 'M2', 'M5', 'M3',
           'DeviceType',
           'DeviceInfo',
           'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19',
           'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27',
           'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35',
           'id_36', 'id_37', 'id_38']

In [7]:
te_tr_new.rename(columns=lambda col: col.replace('id-', 'id_'), inplace=True)

In [8]:
for col in cat_var:
    tr_tr_new[col] = tr_tr_new[col].astype(str)
    te_tr_new[col] = te_tr_new[col].astype(str)

In [9]:
from sklearn.model_selection import train_test_split
X = tr_tr_new.drop('isFraud', axis=1)
y = tr_tr_new['isFraud']

In [10]:
# Split the data into training and temp sets (80% train, 20% temp)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the temp data into validation and test sets (50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 472432
Validation set size: 59054
Test set size: 59054


In [11]:
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import itertools

In [13]:
params_grid = {
    'iterations': [1000,2000],
    'learning_rate': [0.05, 0.07],
    'depth': [8,12],
    'loss_function': ['Logloss'],
    'metric_period': [300]
}

# Initialize a DataFrame to store results
cv_results = pd.DataFrame(columns=['iterations', 'learning_rate', 'depth','loss_function', 'metric_period','mean_auc'])


In [14]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
num_iter = 0

for params in itertools.product(*params_grid.values()):
    # Unpack the current parameter set
    iterations, learning_rate, depth, loss_function, metric_period = params
    
    aucs = []
    for train_index, val_index in kf.split(X):
        X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
        y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
        
        model = CatBoostClassifier(
            iterations=iterations,
            learning_rate=learning_rate,
            depth=depth,
            eval_metric='AUC',
            task_type='GPU',
            cat_features=cat_var,
            random_seed=42,
            verbose=True
        )
        
        model.fit(X_train_fold, y_train_fold)
        preds_proba = model.predict_proba(X_val_fold)[:, 1]
        fold_auc = roc_auc_score(y_val_fold, preds_proba)
        aucs.append(fold_auc)
    
    mean_auc = sum(aucs) / len(aucs)
    cv_results.loc[num_iter] = [iterations, learning_rate, depth, loss_function, metric_period, mean_auc]
    num_iter += 1

print(cv_results)


Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 365ms	remaining: 6m 4s
1:	total: 706ms	remaining: 5m 52s
2:	total: 1.05s	remaining: 5m 50s
3:	total: 1.42s	remaining: 5m 54s
4:	total: 1.79s	remaining: 5m 57s
5:	total: 2.12s	remaining: 5m 50s
6:	total: 2.43s	remaining: 5m 44s
7:	total: 2.75s	remaining: 5m 41s
8:	total: 3.11s	remaining: 5m 42s
9:	total: 3.46s	remaining: 5m 43s
10:	total: 3.81s	remaining: 5m 42s
11:	total: 4.14s	remaining: 5m 40s
12:	total: 4.48s	remaining: 5m 39s
13:	total: 4.81s	remaining: 5m 38s
14:	total: 5.18s	remaining: 5m 40s
15:	total: 5.54s	remaining: 5m 40s
16:	total: 5.81s	remaining: 5m 35s
17:	total: 6.13s	remaining: 5m 34s
18:	total: 6.42s	remaining: 5m 31s
19:	total: 6.76s	remaining: 5m 31s
20:	total: 7.08s	remaining: 5m 29s
21:	total: 7.4s	remaining: 5m 28s
22:	total: 7.76s	remaining: 5m 29s
23:	total: 8.1s	remaining: 5m 29s
24:	total: 8.4s	remaining: 5m 27s
25:	total: 8.74s	remaining: 5m 27s
26:	total: 9.06s	remaining: 5m 26s
27:	total: 9.38s	remaining: 5m 25s
28:	total: 9.68s	remaining: 5m 24s

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 343ms	remaining: 5m 42s
1:	total: 661ms	remaining: 5m 29s
2:	total: 980ms	remaining: 5m 25s
3:	total: 1.3s	remaining: 5m 23s
4:	total: 1.63s	remaining: 5m 24s
5:	total: 1.94s	remaining: 5m 21s
6:	total: 2.26s	remaining: 5m 20s
7:	total: 2.58s	remaining: 5m 19s
8:	total: 2.9s	remaining: 5m 19s
9:	total: 3.22s	remaining: 5m 18s
10:	total: 3.58s	remaining: 5m 21s
11:	total: 3.89s	remaining: 5m 20s
12:	total: 4.21s	remaining: 5m 19s
13:	total: 4.54s	remaining: 5m 19s
14:	total: 4.88s	remaining: 5m 20s
15:	total: 5.2s	remaining: 5m 19s
16:	total: 5.52s	remaining: 5m 19s
17:	total: 5.82s	remaining: 5m 17s
18:	total: 6.18s	remaining: 5m 19s
19:	total: 6.53s	remaining: 5m 20s
20:	total: 6.85s	remaining: 5m 19s
21:	total: 7.12s	remaining: 5m 16s
22:	total: 7.46s	remaining: 5m 16s
23:	total: 7.76s	remaining: 5m 15s
24:	total: 8.1s	remaining: 5m 15s
25:	total: 8.41s	remaining: 5m 15s
26:	total: 8.67s	remaining: 5m 12s
27:	total: 8.95s	remaining: 5m 10s
28:	total: 9.21s	remaining: 5m 8s


Default metric period is 5 because AUC is/are not implemented for GPU


...

0:	total: 739ms	remaining: 24m 36s
1:	total: 1.43s	remaining: 23m 52s
2:	total: 2.21s	remaining: 24m 33s
3:	total: 2.93s	remaining: 24m 21s
4:	total: 3.66s	remaining: 24m 20s
5:	total: 4.41s	remaining: 24m 26s
6:	total: 5.06s	remaining: 24m
7:	total: 5.74s	remaining: 23m 50s
8:	total: 6.46s	remaining: 23m 49s
9:	total: 7.17s	remaining: 23m 47s
10:	total: 7.87s	remaining: 23m 42s
11:	total: 8.63s	remaining: 23m 49s
12:	total: 9.32s	remaining: 23m 45s
13:	total: 9.94s	remaining: 23m 29s
14:	total: 10.5s	remaining: 23m 14s
15:	total: 11.2s	remaining: 23m 10s
16:	total: 11.8s	remaining: 23m
17:	total: 12.5s	remaining: 23m
18:	total: 13.1s	remaining: 22m 50s
19:	total: 13.8s	remaining: 22m 49s
20:	total: 14.4s	remaining: 22m 41s
21:	total: 15.1s	remaining: 22m 33s
22:	total: 15.7s	remaining: 22m 30s
23:	total: 16.3s	remaining: 22m 22s
24:	total: 16.9s	remaining: 22m 18s
25:	total: 17.5s	remaining: 22m 12s
26:	total: 18.2s	remaining: 22m 7s
27:	total: 18.7s	remaining: 21m 59s
28:	total: 19.4

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 734ms	remaining: 24m 26s
1:	total: 1.5s	remaining: 24m 54s
2:	total: 2.19s	remaining: 24m 21s
3:	total: 2.9s	remaining: 24m 9s
4:	total: 3.66s	remaining: 24m 19s
5:	total: 4.36s	remaining: 24m 9s
6:	total: 5.04s	remaining: 23m 55s
7:	total: 5.71s	remaining: 23m 40s
8:	total: 6.37s	remaining: 23m 28s
9:	total: 7.08s	remaining: 23m 28s
10:	total: 7.82s	remaining: 23m 33s
11:	total: 8.5s	remaining: 23m 28s
12:	total: 9.23s	remaining: 23m 30s
13:	total: 9.9s	remaining: 23m 24s
14:	total: 10.6s	remaining: 23m 19s
15:	total: 11.3s	remaining: 23m 19s
16:	total: 12s	remaining: 23m 15s
17:	total: 12.5s	remaining: 23m 1s
18:	total: 13.2s	remaining: 22m 51s
19:	total: 13.8s	remaining: 22m 44s
20:	total: 14.4s	remaining: 22m 38s
21:	total: 15s	remaining: 22m 25s
22:	total: 15.6s	remaining: 22m 24s
23:	total: 16.2s	remaining: 22m 17s
24:	total: 16.9s	remaining: 22m 11s
25:	total: 17.5s	remaining: 22m 9s
26:	total: 18.1s	remaining: 22m
27:	total: 18.7s	remaining: 22m
28:	total: 19.4s	remai

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 724ms	remaining: 24m 6s
1:	total: 1.47s	remaining: 24m 26s
2:	total: 2.19s	remaining: 24m 20s
3:	total: 2.86s	remaining: 23m 49s
4:	total: 3.61s	remaining: 24m 1s
5:	total: 4.33s	remaining: 24m
6:	total: 5.02s	remaining: 23m 50s
7:	total: 5.77s	remaining: 23m 57s
8:	total: 6.47s	remaining: 23m 52s
9:	total: 7.17s	remaining: 23m 47s
10:	total: 7.92s	remaining: 23m 51s
11:	total: 8.58s	remaining: 23m 41s
12:	total: 9.31s	remaining: 23m 42s
13:	total: 9.98s	remaining: 23m 36s
14:	total: 10.5s	remaining: 23m 14s
15:	total: 11.2s	remaining: 23m 13s
16:	total: 11.9s	remaining: 23m 4s
17:	total: 12.5s	remaining: 23m 1s
18:	total: 13.1s	remaining: 22m 46s
19:	total: 13.8s	remaining: 22m 45s
20:	total: 14.4s	remaining: 22m 40s
21:	total: 15.1s	remaining: 22m 33s
22:	total: 15.7s	remaining: 22m 29s
23:	total: 16.3s	remaining: 22m 22s
24:	total: 16.9s	remaining: 22m 17s
25:	total: 17.6s	remaining: 22m 15s
26:	total: 18.2s	remaining: 22m 13s
27:	total: 18.9s	remaining: 22m 9s
28:	total: 

In [15]:
best_params = cv_results.sort_values(by='mean_auc', ascending=False).iloc[0]

print(best_params)

iterations           2000
learning_rate        0.07
depth                  12
loss_function     Logloss
metric_period         300
mean_auc         0.976738
Name: 7, dtype: object


In [12]:
best_params['depth']

7

In [16]:
best_model = CatBoostClassifier(
    iterations=int(best_params['iterations']),
    learning_rate=float(best_params['learning_rate']),
    depth=int(best_params['depth']),
    cat_features=cat_var,
    eval_metric='AUC',  # Changed metric to AUC
    random_seed=42,
    # loss_function=best_params['loss_function'],
    metric_period=int(best_params['metric_period']),
    task_type='GPU'
)

In [ ]:
joblib.dump(best_model, '../models_libjob/cb_model_.joblib')

In [17]:
best_model.fit(
    X_train, y_train,
    cat_features=cat_var,
    eval_set=(X_val, y_val),
    logging_level='Verbose',  # Output to stdout
    plot=True  # Plotting training and validation error (if you run this notebook locally)
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.8248527	best: 0.8248527 (0)	total: 800ms	remaining: 26m 39s
300:	test: 0.9649580	best: 0.9649580 (300)	total: 3m 19s	remaining: 18m 44s
600:	test: 0.9688764	best: 0.9688764 (600)	total: 6m 17s	remaining: 14m 38s
900:	test: 0.9715973	best: 0.9715973 (900)	total: 9m 14s	remaining: 11m 15s
1200:	test: 0.9727079	best: 0.9727079 (1200)	total: 12m 11s	remaining: 8m 6s
1500:	test: 0.9737741	best: 0.9737741 (1500)	total: 15m 10s	remaining: 5m 2s
1800:	test: 0.9742383	best: 0.9742383 (1800)	total: 18m 12s	remaining: 2m
1999:	test: 0.9741523	best: 0.9742383 (1800)	total: 20m 9s	remaining: 0us
bestTest = 0.9742383361
bestIteration = 1800
Shrink model to first 1801 iterations.


In [18]:
joblib.dump(best_model, '../models_libjob/cb[0.9742].joblib')

['../models_libjob/cb[0.9742].joblib']

In [19]:
pred_train_p = best_model.predict_proba(X_train)
pred_val_p = best_model.predict_proba(X_val)
pred_test_p = best_model.predict_proba(X_test)
# Calcular métricas de evaluación
auc_train = roc_auc_score(y_train, pred_train_p[:,1])
auc_val = roc_auc_score(y_val, pred_val_p[:,1])
auc_test = roc_auc_score(y_test, pred_test_p[:,1])
# print error
print('Metric train = %.4f - Metric val = %.4f - Metric test = %.4f'
      % (auc_train, auc_val, auc_test))

Metric train = 0.9985 - Metric val = 0.9742 - Metric test = 0.9789


In [ ]:
joblib.dump(best_model, '../models_libjob/cb[0.9900].joblib')

In [20]:
pred_te_tr_new_p = best_model.predict_proba(te_tr_new)[:, 1]

# Create the output DataFrame
output_df = pd.DataFrame({
    'TransactionID': te_tr_new.reset_index()['TransactionID'],
    'isFraud': pred_te_tr_new_p
})

# Save the DataFrame to a CSV file
output_df.to_csv('predicted_fraud_catboost[6].csv', index=False)


In [ ]:
joblib.dump(best_model, '../models_libjob/cb[0.9316][0.9087].joblib')